In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ensemble/weighted_ensemble.pkl
/kaggle/input/test-pss5e1/test_pss5e1.csv
/kaggle/input/playground-series-s5e1/sample_submission.csv
/kaggle/input/playground-series-s5e1/train.csv
/kaggle/input/playground-series-s5e1/test.csv
/kaggle/input/train-pss5e1/train_pss5e1.csv


In [2]:
!pip install autogluon.tabular[0]
# !pip install autogluon.timeseries[0]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 25.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [3]:
train = pd.read_csv('/kaggle/input/train-pss5e1/train_pss5e1.csv')
test = pd.read_csv('/kaggle/input/test-pss5e1/test_pss5e1.csv')
train = train.dropna(subset='num_sold')

In [4]:
train['time_id'] = train['country'] + '_' + train['product'] + '_' + train['store']
test['time_id'] = test['country'] + '_' + test['product'] + '_' + test['store']

In [5]:
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

In [6]:
train

,id,date,country,store,product,num_sold,year,day_of_month,days_in_month,month,...,day_of_year_sin,day_of_year_cos,quarter_sin,quarter_cos,month_sin,month_cos,country_code,gdp,gdp_ratio,time_id
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0,2010,1,31,1,...,0.017213,0.999852,1.000000e+00,6.123234e-17,5.000000e-01,0.866025,CAN,47560.666601,0.178301,Canada_Kaggle_Discount Stickers
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0,2010,1,31,1,...,0.017213,0.999852,1.000000e+00,6.123234e-17,5.000000e-01,0.866025,CAN,47560.666601,0.178301,Canada_Kaggle Tiers_Discount Stickers
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0,2010,1,31,1,...,0.017213,0.999852,1.000000e+00,6.123234e-17,5.000000e-01,0.866025,CAN,47560.666601,0.178301,Canada_Kerneler_Discount Stickers
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0,2010,1,31,1,...,0.017213,0.999852,1.000000e+00,6.123234e-17,5.000000e-01,0.866025,CAN,47560.666601,0.178301,Canada_Kerneler Dark Mode_Discount Stickers
5,5,2010-01-01,Canada,Stickers for Less,Holographic Goose,300.0,2010,1,31,1,...,0.017213,0.999852,1.000000e+00,6.123234e-17,5.000000e-01,0.866025,CAN,47560.666601,0.178301,Canada_Holographic Goose_Stickers for Less
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230125,230125,2016-12-31,Singapore,Premium Sticker Mart,Holographic Goose,466.0,2016,31,31,12,...,0.017213,0.999852,-2.449294e-16,1.000000e+00,-2.449294e-16,1.000000,SGP,56899.918181,0.231100,Singapore_Holographic Goose_Premium Sticker Mart
230126,230126,2016-12-31,Singapore,Premium Sticker Mart,Kaggle,2907.0,2016,31,31,12,...,0.017213,0.999852,-2.449294e-16,1.000000e+00,-2.449294e-16,1.000000,SGP,56899.918181,0.231100,Singapore_Kaggle_Premium Sticker Mart
230127,230127,2016-12-31,Singapore,Premium Sticker Mart,Kaggle Tiers,2299.0,2016,31,31,12,...,0.017213,0.999852,-2.449294e-16,1.000000e+00,-2.449294e-16,1.000000,SGP,56899.918181,0.231100,Singapore_Kaggle Tiers_Premium Sticker Mart
230128,230128,2016-12-31,Singapore,Premium Sticker Mart,Kerneler,1242.0,2016,31,31,12,...,0.017213,0.999852,-2.449294e-16,1.000000e+00,-2.449294e-16,1.000000,SGP,56899.918181,0.231100,Singapore_Kerneler_Premium Sticker Mart


In [7]:
target = 'num_sold'
features = [col for col in train.columns if (col!='num_sold' and col!='date' and col!='id')]
cols_to_drop = ['date', 'id']
train.drop(columns=cols_to_drop, inplace=True)
test.drop(columns=cols_to_drop, inplace=True)

In [8]:
from scipy.stats import boxcox
train['num_sold'], lmbda = boxcox(train['num_sold'])

In [9]:
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label=target, eval_metric='mean_absolute_percentage_error', problem_type='regression')

# tabular_df = TimeSeriesDataFrame(train)
# label = 'num_sold'
# features = [col for col in tabular_df.columns if col!='num_sold']
# aml = TimeSeriesPredictor(label=label, eval_metric='mean_absolute_percentage_error', problem_type='regression')

No path specified. Models will be saved in: "AutogluonModels/ag-20250125_111627"


In [10]:
len(train.columns)

37

In [11]:
predictor.fit(
    train_data=train,
    time_limit=39600,
    presets='best_quality',
    num_bag_folds=10,
    hyperparameters={
        'GBM': {'num_boost_round': 10000, 'tree_method': 'gpu_hist', 'device': 'gpu'},  # LightGBM
        'CAT': {'task_type': 'GPU'},  # CatBoost
        'XGB': {'tree_method': 'gpu_hist', 'gpu_id': 0},  # XGBoost
        'NN_TORCH': {'use_gpu': True},  # Neural networks with GPU

    },
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       30.04 GB / 31.35 GB (95.8%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=10, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked

[1000]	valid_set's l2: 0.346445	valid_set's mean_absolute_percentage_error: -0.019409
[2000]	valid_set's l2: 0.332618	valid_set's mean_absolute_percentage_error: -0.018991
[3000]	valid_set's l2: 0.329886	valid_set's mean_absolute_percentage_error: -0.0189074
[4000]	valid_set's l2: 0.329645	valid_set's mean_absolute_percentage_error: -0.0188838
[5000]	valid_set's l2: 0.33064	valid_set's mean_absolute_percentage_error: -0.0189004
[1000]	valid_set's l2: 0.338388	valid_set's mean_absolute_percentage_error: -0.0194858
[2000]	valid_set's l2: 0.32498	valid_set's mean_absolute_percentage_error: -0.0191338
[3000]	valid_set's l2: 0.322217	valid_set's mean_absolute_percentage_error: -0.0190162
[4000]	valid_set's l2: 0.321595	valid_set's mean_absolute_percentage_error: -0.018966
[5000]	valid_set's l2: 0.322302	valid_set's mean_absolute_percentage_error: -0.0189651
[1000]	valid_set's l2: 0.342949	valid_set's mean_absolute_percentage_error: -0.0194918
[2000]	valid_set's l2: 0.329122	valid_set's mean

	-0.0189	 = Validation score   (-mean_absolute_percentage_error)
	337.81s	 = Training   runtime
	17.63s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 6234.41s of the 9533.20s of remaining time.
	Fitting 10 child models (S1F1 - S1F10) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0244	 = Validation score   (-mean_absolute_percentage_error)
	685.09s	 = Training   runtime
	2.91s	 = Validation runtime
Fitting model: XGBoost_BAG_L1 ... Training model for up to 5545.95s of the 8844.74s of remaining time.
	Fitting 10 child models (S1F1 - S1F10) | Fitting with SequentialLocalFoldFittingStrategy
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [11:34:05] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [11:34:05] WARNING: /workspace/src/

[1000]	valid_set's l2: 0.336724	valid_set's mean_absolute_percentage_error: -0.0193734
[2000]	valid_set's l2: 0.327188	valid_set's mean_absolute_percentage_error: -0.0190324
[3000]	valid_set's l2: 0.325145	valid_set's mean_absolute_percentage_error: -0.0189589
[4000]	valid_set's l2: 0.325475	valid_set's mean_absolute_percentage_error: -0.0189526
[5000]	valid_set's l2: 0.326644	valid_set's mean_absolute_percentage_error: -0.0189539
[6000]	valid_set's l2: 0.327797	valid_set's mean_absolute_percentage_error: -0.0189673
[1000]	valid_set's l2: 0.34376	valid_set's mean_absolute_percentage_error: -0.0193683
[2000]	valid_set's l2: 0.32985	valid_set's mean_absolute_percentage_error: -0.0189505
[3000]	valid_set's l2: 0.328471	valid_set's mean_absolute_percentage_error: -0.0188648
[4000]	valid_set's l2: 0.32927	valid_set's mean_absolute_percentage_error: -0.0188639
[1000]	valid_set's l2: 0.336269	valid_set's mean_absolute_percentage_error: -0.0194571
[2000]	valid_set's l2: 0.323529	valid_set's me

	-0.0189	 = Validation score   (-mean_absolute_percentage_error)
	312.93s	 = Training   runtime
	34.84s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 25007.54s of the 37696.14s of remaining time.
	Fitting 10 child models (S1F1 - S1F10) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0191	 = Validation score   (-mean_absolute_percentage_error)
	1075.34s	 = Training   runtime
	4.58s	 = Validation runtime
Fitting model: XGBoost_BAG_L1 ... Training model for up to 23927.05s of the 36615.65s of remaining time.
	Fitting 10 child models (S1F1 - S1F10) | Fitting with SequentialLocalFoldFittingStrategy
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:06:14] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python

In [12]:
leaderboard = predictor.leaderboard()
leaderboard.to_csv('leaderboard.csv', index=False)

In [13]:
leaderboard

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-0.018492,mean_absolute_percentage_error,45.409579,1755.288345,0.004287,0.349548,3,True,8
1,WeightedEnsemble_L2,-0.018581,mean_absolute_percentage_error,42.646244,1520.315774,0.003919,0.177280,2,True,4
2,XGBoost_BAG_L2,-0.018603,mean_absolute_percentage_error,43.642680,1547.157800,1.000355,27.019306,2,True,7
3,LightGBM_BAG_L2,-0.018626,mean_absolute_percentage_error,43.930616,1556.049509,1.288291,35.911015,2,True,5
4,LightGBM_BAG_L1,-0.018875,mean_absolute_percentage_error,34.835170,312.931865,34.835170,312.931865,1,True,1
5,XGBoost_BAG_L1,-0.019034,mean_absolute_percentage_error,3.229614,131.870789,3.229614,131.870789,1,True,3
6,CatBoost_BAG_L1,-0.019088,mean_absolute_percentage_error,4.577541,1075.335840,4.577541,1075.335840,1,True,2
7,CatBoost_BAG_L2,-0.019449,mean_absolute_percentage_error,43.116646,1692.008476,0.474321,171.869982,2,True,6


In [14]:
predictions = predictor.predict(test)
predictions

0        19.323559
1        30.072632
2        27.984558
3        21.619816
4        23.280861
           ...    
98545    21.191885
98546    43.842415
98547    40.399117
98548    32.118626
98549    34.800621
Name: num_sold, Length: 98550, dtype: float32

In [15]:
def inverse_boxcox(y_transformed, lmbda):
    if lmbda==0:
        return np.exp(y_transformed)
    else:
        return np.power(y_transformed*lmbda+1, 1/lmbda)
# preds = inverse_quantile(preds)
preds = inverse_boxcox(predictions, lmbda)

In [16]:
sample_test = pd.read_csv('/kaggle/input/playground-series-s5e1/test.csv')
idx = sample_test['id']

In [17]:
predictions_df = pd.DataFrame({
    'id':idx,
    'num_sold':preds
})
predictions_df.to_csv('submission.csv', index=False)